Для решения данной задачи я решил использовать матричную факторизацию, а именно алгоритм ALS
Я взял train.csv и преобразовал его в user-item матрицу, где 0 значит что пользовательне слушал эту песню, 1 если слушал, и 2 если train.target == 1, т.е. пользователь переслушивал свою песню в течение месяца (это признак того что она ему понравилась).

In [1]:
!pip install implicit

In [2]:
import pandas as pd
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score

In [3]:
# !7z x ./train.csv.7z

In [41]:
data = pd.read_csv('train.csv', usecols=['msno', 'song_id', 'target'])

In [42]:
len(data.msno.unique()), len(data.song_id.unique()), len(data)

(30755, 359966, 7377418)

Видим, что данных очень много и google colab просто упадет (проверено эксперементально), поэтому я оставлю информацию только о 200 самых популярных песен. С одной стороны, мы теряем в качестве рекомендаций, с другой же - нам в любом случае выгоднее рекомендовать более популрные треки

In [43]:
N = 200

In [44]:
top_songs = data['song_id'].value_counts().nlargest(N)

In [45]:
data = data[data.song_id.isin(top_songs.index)]
data.set_index('msno', inplace=True)
data

,song_id,target
msno,,
Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,VkILU0H1h3NMmk9MQrXouNudGk5n8Ls5cqRRuBxeTh4=,1
TJU0Gfvy7FB+r89bWovPKXTjuApTCiv3xg/tt5shR78=,u6/Pb7X4u7KU4gXrBgGqt8RlRrNNFLn03tLAHyxRxwA=,0
TJU0Gfvy7FB+r89bWovPKXTjuApTCiv3xg/tt5shR78=,IgMar/mVrJQ+ODFPytDf7jwQMmR09+slyZUdVylRFLc=,1
0LhkakIQDn36HZXI6ClQSO7W7jkpZAy+9MvYgPOZGrA=,DLBDZhOoW7zd7GBV99bi92ZXYUS26lzV+jJKbHshP5c=,1
0LhkakIQDn36HZXI6ClQSO7W7jkpZAy+9MvYgPOZGrA=,T86YHdD4C9JSc274b1IlMkLuNdz4BQRB50fWWE7hx9g=,1
...,...,...
I8plKBAmVLQLkm1smbhFRxuSg1Wt1XCAWQcXPCUgDFA=,SHV52/0W8kZcf3uJZF9+ZP3S5vu2RO4WNGhZ8qQHulU=,1
jnWlUt2FGlgchWH6TWGggWnrxd969y4LyzEgstf4VwU=,Ce8Mui+yVruumWy9pKMDfSRBFg1m93m/+XR8FHlktaE=,1
rn5ULZY2LVxhTqK9UojaY6COgmu9aYq06CTiOL3KLIE=,oTGg75yjTAP0grXmGld3EUD0b/b9u7lW2JyUX/mBiXI=,0


Теперь переведем колонку `song_ig` в onehot представление и учтем занчения `target`

In [46]:
data = pd.get_dummies(data, columns=['song_id'], prefix='', prefix_sep='', dtype=int)

for col in data.columns[1:]:
    data[col] = data[col] * (1 + data.target)

data = data.groupby(level=0).sum()
data.drop('target', inplace=True, axis=1)

data

,+CvzJ0dbjOAypszTDhMrSjJJQYNbEiDrSod3wwcOA1k=,+LztcJcPEEwsikk6+K5udm06XJQMzR4+lzavKLUyE0k=,+Sm75wnBf/sjm/QMUAFx8N+Ae04kWCXGlgH50tTeM6c=,+SstqMwhQPBQFTPBhLKPT642IiBDXzZFwlzsLl4cGXo=,+cP03fE+hnJUB/v2PHLsD9UHBjmDIYvbuX+NrFqeudI=,+h+uIXLQyXHuCdwAY5oT3T302vt0IdQJtux0APXhriw=,/6kgyFt1nsLDNbZh0AHM4Pb3sbp3CZ0C4DbPa4++eVE=,/70HjygVDhHsKBoV8mmsBg/WduSgs4+Zg6GfzhUQbdk=,/RYwZD+IL/0YEgpu71HgW9q6c2A80gr/fC/j0BmYGvo=,0b8M2FLEH176y/ng1rK9ftUYDYnA+1NdoCOUeuyBMtU=,...,xK0clTH8TvjG6sKmJXZuMLzhikhlEsolLtx2AHBpmbQ=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,yPA+sQt5tJ3NgR6/L9BefmNpA2152zQQ+/Kvtt3iSUs=,ys+EL8Sok4HC4i7sDY0+slDNGVZ8+uOQi6TQ6g8VSF4=,zHqZ07gn+YvF36FWzv9+y8KiCMhYhdAUS+vSIKY3UZY=,zQ42nV6yi4mJlCn/6KEjm+SP6pEwUsd3P9fpI8tDiWE=,ze0+7p13wqB1hZ/KWd6OT42LBehKmYVQyGGzEv8Q4fA=,zg6SzD3aqsnOqqyhFYTdbCNFrhSACbwefB/ZFi68AO0=,zuyzUP11fbgoVp6IMfWK378GDToe59ZlK5gkf+7aKdg=,zx4aCmk+b/uPmvESsG8ekvi7qWmifSPNMXg1i8wyG8Q=
msno,,,,,,,,,,,,,,,,,,,,,
++5wYjoMgQHoRuD3GbbvmphZbBBwymzv5Q4l8sywtuU=,0,0,2,0,0,0,2,0,0,0,...,0,1,0,0,2,0,0,0,2,0
++AH7m/EQ4iKe6wSlfO/xXAJx50p+fCeTyF90GoE9Pg=,0,0,0,2,0,2,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
++e+jsxuQ8UEnmW40od9Rq3rW7+wAum4wooXyZTKJpk=,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
++xWL5Pbi2CpG4uUugigQahauM0J/sBIRloTNPBybIU=,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
+/SKX44s4ryWQzYzuV7ZKMXqIKQMN1cPz3M8CJ8CFKU=,0,2,0,2,0,2,2,2,0,0,...,0,0,0,2,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzTnhToV1XfDIsDJ1YaXM3Q05xV54T8Q0zTJkakXsks=,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zzZBJUYXrb168A4Ff4sA8L2iOH0x4ciKdD6WkV53XaE=,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
zzompfSaMamqvjyCMYvgUBwYrxh8fHE40z3f73CQoak=,0,0,0,0,0,1,0,0,0,0,...,0,0,0,2,1,0,0,0,0,0


Делим данные на train/test и обучаем модель

In [47]:
train, test = train_test_split(data, test_size=0.2)
csr_train = csr_matrix(train)
csr_test = csr_matrix(test)

In [48]:
model = AlternatingLeastSquares(factors=10, iterations=30, calculate_training_loss=True)
model.fit(csr_train)

  0%|          | 0/30 [00:00<?, ?it/s]

Теперь оценим работу модели на тестовой выборке путем усреднения `ndcg_score` на пользователях

In [49]:
score_sum = 0

for idx in range(len(test)):
    indices, scores = model.recommend(idx, csr_test[idx], N=20)
    user = test.loc[test.index[idx]]
    true_values = user.values[indices]
    score_sum += ndcg_score(true_values[None, :], scores[None, :], k=20)


print('average NDCG@20 score:', score_sum / len(test))

average NDCG@20 score: 0.0023223307797706153


Вывод - матричная факторизация на основе только прослушиваний пользователей показала плохое качество работы. К сожалению, рекомендательные системы не являются моей сильной стороной, и это все что я смог сделать :)